In [ ]:
import sys
import os


repo_root = os.path.abspath("..")   
sys.path.append(repo_root)

from src.feature_engineering import build_player_team_map_from_events, compute_features_for_all_events, parse_match_id_from_unique, compute_snapshot_features_for_event, build_frames_dict, group_tracking_by_match_and_frame, nearest_available_frame
import pandas as pd


In [2]:
events = pd.read_csv("../Our Datasets/processed_recoverys_and_interceptions_dynamic_events.csv")
tracking = pd.read_csv("../Our Datasets/processed_tracking_data_start.csv")


In [3]:
events.shape; tracking.shape
events.columns[:20]
events.head(3).T


,0,1,2
phase_index,6,6,7
player_possession_phase_index,1.0,2.0,1.0
event_type,player_possession,player_possession,player_possession
event_subtype,NaN,NaN,NaN
start_type,recovery,pass_reception,pass_reception
end_type,pass,pass,pass
pass_outcome,successful,successful,unsuccessful
player_in_possession_name,NaN,NaN,NaN
player_in_possession_id,NaN,NaN,NaN
team_shortname,Newcastle,Newcastle,Newcastle


In [4]:
sample_unique = events['Unique ID'].dropna().iloc[0]
match_id = parse_match_id_from_unique(sample_unique)


In [5]:
ev_match = events[events['Unique ID'].str.startswith(str(match_id))]
tr_match = tracking[tracking['match_id']==match_id]


In [ ]:
ev_row = ev_match.iloc[5]
feat = compute_snapshot_features_for_event(ev_row, build_frames_dict(tr_match, match_id), sorted(tr_match['frame'].unique()), build_player_team_map_from_events(events, match_id))
feat


{'Unique ID': '1886347_170_193',
 'match_id': 1886347,
 'event_index': 177,
 'frame_anchor': 1492,
 'rec_player_id': 735574,
 'rec_team_short': 'Auckland FC',
 'dist_to_near_goal': 34.42740332932473,
 'dist_to_far_goal': 75.41648427233929,
 'd_nearest_opp': 7.359347797189642,
 'n_opp_within5': 0,
 'd_nearest_team': 16.05252005138134,
 'mean_team_dist': 24.542266274619536,
 'n_forward_options': 6,
 'game_state': 'drawing',
 'team_in_possession_phase_type': None,
 'team_out_of_possession_phase_type': 'chaotic',
 'third_start': 'attacking',
 'third_end': 'attacking',
 'start_type': nan,
 'end_type': nan,
 'max_player_targeted_xthreat': 0.0091}

In [ ]:
feats_df = compute_features_for_all_events(
    "../Our Datasets/processed_recoverys_and_interceptions_dynamic_events.csv",
    "../Our Datasets/processed_tracking_data_start.csv"
)
  
feats_df.head()


Loading events...
Loading tracking snapshot...
Found 575 rows with errors (player/frame issues). They will remain flagged.


,Unique ID,match_id,event_index,frame_anchor,rec_player_id,rec_team_short,dist_to_near_goal,dist_to_far_goal,d_nearest_opp,n_opp_within5,...,team_in_possession_phase_type,team_out_of_possession_phase_type,third_start,third_end,start_type,end_type,max_player_targeted_xthreat,event_row_index,source_file,error
0,1886347_42_46,1886347.0,42.0,387.0,735578.0,Newcastle,10.301189,99.575170,7.026308,0.0,...,NaN,high_block,defensive,defensive,recovery,pass,0.0024,0,1886347.0,NaN
1,1886347_42_46,1886347.0,43.0,387.0,51649.0,Newcastle,4.830041,100.170002,6.743923,0.0,...,NaN,high_block,defensive,defensive,pass_reception,pass,0.0024,1,1886347.0,NaN
2,1886347_42_46,1886347.0,46.0,387.0,966120.0,Newcastle,49.353746,57.945597,0.000000,1.0,...,NaN,high_block,middle,middle,pass_reception,pass,0.0024,2,1886347.0,NaN
3,1886347_170_193,1886347.0,170.0,1492.0,735574.0,Newcastle,34.427403,75.416484,0.000000,1.0,...,NaN,chaotic,attacking,attacking,recovery,pass,0.0091,3,1886347.0,NaN
4,1886347_170_193,1886347.0,174.0,1492.0,735573.0,Newcastle,16.621107,96.316464,2.772003,1.0,...,NaN,chaotic,attacking,attacking,pass_reception,pass,0.0091,4,1886347.0,NaN


In [8]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
feats_df.head()

,Unique ID,match_id,event_index,frame_anchor,rec_player_id,rec_team_short,dist_to_near_goal,dist_to_far_goal,d_nearest_opp,n_opp_within5,d_nearest_team,mean_team_dist,n_forward_options,game_state,team_in_possession_phase_type,team_out_of_possession_phase_type,third_start,third_end,start_type,end_type,max_player_targeted_xthreat,event_row_index,source_file,error
0,1886347_42_46,1886347.0,42.0,387.0,735578.0,Newcastle,10.301189,99.575170,7.026308,0.0,8.593788,31.734731,10.0,drawing,NaN,high_block,defensive,defensive,recovery,pass,0.0024,0,1886347.0,NaN
1,1886347_42_46,1886347.0,43.0,387.0,51649.0,Newcastle,4.830041,100.170002,6.743923,0.0,2.240759,28.398055,11.0,drawing,NaN,high_block,defensive,defensive,pass_reception,pass,0.0024,1,1886347.0,NaN
2,1886347_42_46,1886347.0,46.0,387.0,966120.0,Newcastle,49.353746,57.945597,0.000000,1.0,3.590125,29.727513,3.0,drawing,NaN,high_block,middle,middle,pass_reception,pass,0.0024,2,1886347.0,NaN
3,1886347_170_193,1886347.0,170.0,1492.0,735574.0,Newcastle,34.427403,75.416484,0.000000,1.0,7.359348,27.331335,9.0,drawing,NaN,chaotic,attacking,attacking,recovery,pass,0.0091,3,1886347.0,NaN
4,1886347_170_193,1886347.0,174.0,1492.0,735573.0,Newcastle,16.621107,96.316464,2.772003,1.0,11.619509,26.302980,6.0,drawing,NaN,chaotic,attacking,attacking,pass_reception,pass,0.0091,4,1886347.0,NaN


In [9]:
feats_df.tail()

,Unique ID,match_id,event_index,frame_anchor,rec_player_id,rec_team_short,dist_to_near_goal,dist_to_far_goal,d_nearest_opp,n_opp_within5,d_nearest_team,mean_team_dist,n_forward_options,game_state,team_in_possession_phase_type,team_out_of_possession_phase_type,third_start,third_end,start_type,end_type,max_player_targeted_xthreat,event_row_index,source_file,error
5850,2017461_3747_3747,2017461.0,3747.0,65560.0,14736.0,Auckland FC,33.448762,82.279522,5.026659,0.0,4.000562,18.579824,6.0,winning,NaN,chaotic,defensive,defensive,recovery,pass,NaN,5850,2017461.0,NaN
5851,2017461_3782_3782,2017461.0,3782.0,65920.0,50979.0,Melbourne V FC,52.203530,59.568519,6.901311,0.0,2.870627,19.184191,9.0,losing,NaN,chaotic,middle,middle,pass_interception,pass,NaN,5851,2017461.0,NaN
5852,2017461_3837_3837,2017461.0,3837.0,66806.0,4322.0,Auckland FC,34.236218,86.986313,5.721416,0.0,5.820009,21.764702,6.0,winning,NaN,chaotic,attacking,attacking,recovery,pass,0.0133,5852,2017461.0,NaN
5853,2017461_3932_3932,2017461.0,3932.0,67716.0,745320.0,Melbourne V FC,20.692561,87.764356,6.528714,0.0,5.679736,14.118092,8.0,losing,NaN,disruption,attacking,attacking,pass_interception,pass,NaN,5853,2017461.0,NaN
5854,2017461_4095_4095,2017461.0,4095.0,69308.0,38673.0,Auckland FC,36.021022,69.586737,6.412652,0.0,10.006243,27.315183,1.0,winning,NaN,disruption,middle,middle,pass_interception,pass,NaN,5854,2017461.0,NaN


In [ ]:
#feats_df.to_parquet("Our Datasets/features_sample_match.parquet", index=False)
#feats_df.to_csv("Our Datasets/features_sample_match.csv", index=False)